In [2]:
# Supress Warnings

import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np

#Reading companies and rounds2 dataframe
companies = pd.read_csv("./companies.txt",delimiter = "\t",encoding='latin')
rounds2 = pd.read_csv("./rounds2.csv", encoding='latin')


Checkpoint 1: Data Cleaning 1
Load the companies and rounds data (provided on the previous page) into two data frames and name them companies and rounds2 respectively.

In [4]:
rounds2.head()
rounds2.isnull().sum()
round(100*rounds2.isnull().sum()/len(rounds2.index),2)

#Funding_round_code is not being referred to so leaving as it is, in actual it can be removed from the dataframe

company_permalink           0.00
funding_round_permalink     0.00
funding_round_type          0.00
funding_round_code         72.91
funded_at                   0.00
raised_amount_usd          17.39
dtype: float64

In [5]:
companies.shape
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 10 columns):
permalink        66368 non-null object
name             66367 non-null object
homepage_url     61310 non-null object
category_list    63220 non-null object
status           66368 non-null object
country_code     59410 non-null object
state_code       57821 non-null object
region           58338 non-null object
city             58340 non-null object
founded_at       51147 non-null object
dtypes: object(10)
memory usage: 5.1+ MB


In [6]:
companies.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
3,/Organization/0-6-Com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
4,/Organization/004-Technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010


In [7]:
#How many unique companies are there
companies.permalink.describe()

count                   66368
unique                  66368
top       /Organization/Netli
freq                        1
Name: permalink, dtype: object

In [8]:
rounds2.company_permalink.describe()

count                       114949
unique                       90247
top       /ORGANIZATION/SOLARFLARE
freq                            10
Name: company_permalink, dtype: object

In [9]:
#Updating rounds2 with encoding corrections
rounds2['company_permalink'] = rounds2.company_permalink.str.encode('utf-8').str.decode('ascii', 'ignore')

In [10]:
#Clean up rounds2
rounds2.isnull().sum()
#There are 19990 rows around 17% in which NUll value is there, we need to delete these as there is no way, these can be imputed
rounds2 = rounds2[~np.isnan(rounds2['raised_amount_usd'])]
round(100*rounds2.isnull().sum()/len(rounds2.index),2)

company_permalink           0.00
funding_round_permalink     0.00
funding_round_type          0.00
funding_round_code         70.34
funded_at                   0.00
raised_amount_usd           0.00
dtype: float64

In [11]:
rounds2.shape
#Dropping Column funding_round_code as it is having 70% Null values, no use keeping this
rounds2.drop(axis=1, inplace = True, columns=['funding_round_code'])

In [12]:
round(100*rounds2.isnull().sum()/len(rounds2.index),2)

company_permalink          0.0
funding_round_permalink    0.0
funding_round_type         0.0
funded_at                  0.0
raised_amount_usd          0.0
dtype: float64

rounds2 dataframe is cleaned-up, it cane be further used. 
company_permalink is not unique, there are more than 1 entires for a single name means same company is funded again
unique id for each funding round is funding_round_permalink in case to be used later on

In [19]:
print(rounds2.company_permalink.describe())
print(rounds2.shape)
#rounds2.head()
len(rounds2.company_permalink.unique())

count                        94959
unique                       73780
top       /ORGANIZATION/SOLARFLARE
freq                            10
Name: company_permalink, dtype: object
(94959, 5)


73780

In [13]:
#Start clean-up on companies data frame
#Encoding issue correction
companies['permalink'] = companies.permalink.str.encode('utf-8').str.decode('ascii', 'ignore')
round(100*companies.isnull().sum()/len(companies.index),2)

permalink         0.00
name              0.00
homepage_url      7.62
category_list     4.74
status            0.00
country_code     10.48
state_code       12.88
region           12.10
city             12.10
founded_at       22.93
dtype: float64

In [14]:
companies.head()
print(companies.shape)
#There are Null values in companies frame, those will be cleaned up later on as per the need of those values

#Dropping founded_at column as this is not of use and 23% NUll values are there
companies.drop(axis=1, inplace = True, columns=['founded_at'])

(66368, 10)


In [15]:
round(100*companies.isnull().sum()/len(companies.index),2)
#Going ahead now, still there are NUll values in category_list as 1.7%.. to be handled later on

permalink         0.00
name              0.00
homepage_url      7.62
category_list     4.74
status            0.00
country_code     10.48
state_code       12.88
region           12.10
city             12.10
dtype: float64

In [16]:
#Are there any companies in the rounds2 file which are not present in companies? 
#we need to keep company name in lower case in both frames

companies['permalink']=companies['permalink'].apply(lambda x : x.lower())
rounds2['company_permalink'] = rounds2['company_permalink'].apply(lambda x : x.lower())

#Show which companies are not there in companies list
result = rounds2.loc[~rounds2.company_permalink.isin (companies.permalink)]
print(result.head())
print(len(result.index))
#All rounds2 companies are there in companies list

Empty DataFrame
Columns: [company_permalink, funding_round_permalink, funding_round_type, funded_at, raised_amount_usd]
Index: []
0


In [17]:
#Merging the frame here
#Using Left merge as we are to analyze data further regarding companies which are funded
master_frame = pd.merge(rounds2, companies, how='left', left_on='company_permalink', right_on='permalink')

In [18]:
print(rounds2.shape)
print(master_frame.shape)

(94959, 5)
(94959, 14)


In [19]:
master_frame.head()

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai
1,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,01-03-2014,700000.0,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City
2,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,30-01-2014,3406878.0,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN
3,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,19-03-2008,2000000.0,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing
4,/organization/01games-technology,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,01-07-2014,41250.0,/organization/01games-technology,01Games Technology,http://www.01games.hk/,Games,operating,HKG,NaN,Hong Kong,Hong Kong


In [20]:
#Analyze master_frame now
round(100* master_frame.isnull().sum()/len(master_frame.index),2)

company_permalink          0.00
funding_round_permalink    0.00
funding_round_type         0.00
funded_at                  0.00
raised_amount_usd          0.00
permalink                  0.00
name                       0.00
homepage_url               4.56
category_list              1.10
status                     0.00
country_code               6.16
state_code                 8.01
region                     7.42
city                       7.42
dtype: float64

Country code is Null for 6.16% records. We will clean up later on as per the need of analysis.
Dropping permalink column as company_permalink is already there

In [21]:
print(companies.shape)
print(rounds2.shape)

master_frame.drop(axis=1, columns=['permalink'],inplace=True)
#print(master_frame.head(5))

print(master_frame.shape)



(66368, 9)
(94959, 5)
(94959, 13)


In [22]:
master_frame.funding_round_type.unique()

array(['venture', 'seed', 'undisclosed', 'convertible_note',
       'private_equity', 'debt_financing', 'angel', 'grant',
       'equity_crowdfunding', 'post_ipo_equity', 'post_ipo_debt',
       'product_crowdfunding', 'secondary_market',
       'non_equity_assistance'], dtype=object)

In [23]:
#master_frame.info()
#converting raised_amount_usd column units to millions
print(master_frame.raised_amount_usd[10])
master_frame.raised_amount_usd = round(master_frame.raised_amount_usd/1000000,2)
print(master_frame.raised_amount_usd[10])

8900000.0
8.9


In [24]:
#export master_frame
master_frame.to_csv("./master_frame.csv",sep='\t', encoding='utf-8', index=False)

In [25]:
round(100* master_frame.isnull().sum()/len(master_frame.index),2)

company_permalink          0.00
funding_round_permalink    0.00
funding_round_type         0.00
funded_at                  0.00
raised_amount_usd          0.00
name                       0.00
homepage_url               4.56
category_list              1.10
status                     0.00
country_code               6.16
state_code                 8.01
region                     7.42
city                       7.42
dtype: float64

Checkpoint 2: Funding Type Analysis

In [26]:
#We are using here 2 columns raised_amount_usd & funding_round_type.. there are no Null values so using the frame as it is

master_frame.pivot_table(values='raised_amount_usd', aggfunc = 'mean', index = 'funding_round_type')
#master_frame.pivot_table(values='raised_amount_usd', aggfunc = 'mean', index = ['venture', 'seed','private_equity','angel'])

,raised_amount_usd
funding_round_type,
angel,0.958576
convertible_note,1.453338
debt_financing,17.043496
equity_crowdfunding,0.538241
grant,4.300457
non_equity_assistance,0.410959
post_ipo_debt,168.704539
post_ipo_equity,82.182532
private_equity,73.308574


In [27]:
#we need to prepare data 
#Cleaning up is needed
#conditions are to be applied
#Chosen funding type is Venture, let's create the frame having only venture funding type data
master_frame_venture = master_frame[master_frame.funding_round_type == 'venture']
master_frame_venture.head(5)
round(100* master_frame_venture.isnull().sum()/len(master_frame_venture.index),2)
     

company_permalink          0.00
funding_round_permalink    0.00
funding_round_type         0.00
funded_at                  0.00
raised_amount_usd          0.00
name                       0.00
homepage_url               5.19
category_list              1.00
status                     0.00
country_code               4.21
state_code                 5.70
region                     5.40
city                       5.40
dtype: float64

In [28]:
#Here Country_code is Null for 4.2% records, deleting these rows as analysis is based on this value only and value can't be imputed
master_frame_venture = master_frame_venture.loc[~master_frame_venture['country_code'].isnull()]
print(round(100* master_frame_venture.isnull().sum()/len(master_frame_venture.index),2))
master_frame_venture.head(5)
#master_frame_venture looks good to go now as non null values are not going to be used

company_permalink          0.00
funding_round_permalink    0.00
funding_round_type         0.00
funded_at                  0.00
raised_amount_usd          0.00
name                       0.00
homepage_url               4.52
category_list              0.63
status                     0.00
country_code               0.00
state_code                 1.55
region                     1.24
city                       1.24
dtype: float64


,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,name,homepage_url,category_list,status,country_code,state_code,region,city
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10.00,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai
3,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,19-03-2008,2.00,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing
6,/organization/0ndine-biomedical-inc,/funding-round/954b9499724b946ad8c396a57a5f3b72,venture,21-12-2009,0.72,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,operating,CAN,BC,Vancouver,Vancouver
8,/organization/0xdata,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,09-11-2015,20.00,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View
9,/organization/0xdata,/funding-round/ae2a174c06517c2394aed45006322a7e,venture,03-01-2013,1.70,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View


In [29]:
master_frame_venture.isnull().sum()

company_permalink             0
funding_round_permalink       0
funding_round_type            0
funded_at                     0
raised_amount_usd             0
name                          0
homepage_url               2175
category_list               302
status                        0
country_code                  0
state_code                  744
region                      596
city                        596
dtype: int64

Checkpoint 3: Country Analysis
This is the second goal of analysis — country analysis.

In [30]:
newFrame  = master_frame_venture.groupby(['country_code'])

top9Frame = pd.DataFrame(newFrame['raised_amount_usd'].sum().sort_values(ascending=False).head(9))

print(top9Frame)

top9CountriesFrame = master_frame_venture[master_frame_venture['country_code'].isin(list(top9Frame.index))]
print(top9CountriesFrame.shape)
print(master_frame_venture.shape)

              raised_amount_usd
country_code                   
USA                   422510.47
CHN                    39835.29
GBR                    20245.59
IND                    14391.91
CAN                     9583.32
FRA                     7259.50
ISR                     6907.50
DEU                     6347.04
JPN                     3363.70
(44383, 13)
(48111, 13)


In [31]:
#Exporting TopCountriesFrame
top9CountriesFrame.to_csv("./top9CountriesFrame.csv",sep='\t', encoding='utf-8', index=False)

In [32]:
top9CountriesFrame.isnull().sum()

company_permalink             0
funding_round_permalink       0
funding_round_type            0
funded_at                     0
raised_amount_usd             0
name                          0
homepage_url               2070
category_list               265
status                        0
country_code                  0
state_code                  287
region                      449
city                        449
dtype: int64

In [33]:
#English is official language in USA, GBR, IND, CAN but not in CHN as checked by the link provided in assignment manually
top9Frame.head(10)

,raised_amount_usd
country_code,
USA,422510.47
CHN,39835.29
GBR,20245.59
IND,14391.91
CAN,9583.32
FRA,7259.50
ISR,6907.50
DEU,6347.04
JPN,3363.70


Checkpoint-4 Sector Analysis

In [34]:
#Reading mapping.csv
mapping = pd.read_csv("./mapping.csv", encoding='latin')

In [35]:
#Renaming the column for merge simplicity
mapping.rename({'category_list':'primarySector'}, axis=1, inplace=True)
mapping.shape
mapping.head()


,primarySector,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1


#Mapping frame needs to be corrected
1. zero 0 needs to be replaced with na
2. 0 as beginning of word needs to be replaced with Na
3. word having 2.0 needs to be skipped

Top9CountriesFrame has 265 Null records in category_list, these rows are to be deleted as we are now analyzing on the same.


In [36]:
top9CountriesFrame = top9CountriesFrame[~top9CountriesFrame.category_list.isnull()]

In [37]:
mapping.info()
#Removing one Null value from the frame
mapping = mapping[~mapping.primarySector.isnull()] 
mapping.isnull().sum()

mapping[mapping.primarySector.str.contains('0')]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688 entries, 0 to 687
Data columns (total 10 columns):
primarySector                              687 non-null object
Automotive & Sports                        688 non-null int64
Blanks                                     688 non-null int64
Cleantech / Semiconductors                 688 non-null int64
Entertainment                              688 non-null int64
Health                                     688 non-null int64
Manufacturing                              688 non-null int64
News, Search and Messaging                 688 non-null int64
Others                                     688 non-null int64
Social, Finance, Analytics, Advertising    688 non-null int64
dtypes: int64(9), object(1)
memory usage: 53.8+ KB


,primarySector,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
20,Alter0tive Medicine,0,0,0,0,1,0,0,0,0
22,A0lytics,0,0,0,0,0,0,0,0,1
59,Big Data A0lytics,0,0,0,0,0,0,0,0,1
79,Business A0lytics,0,0,0,0,0,0,0,0,1
89,Can0bis,0,0,0,0,1,0,0,0,0
91,Career Ma0gement,0,0,0,0,0,0,0,0,1
103,Chi0 Internet,0,0,0,0,0,0,1,0,0
114,Cloud Ma0gement,0,0,0,0,0,0,1,0,0
145,Contact Ma0gement,0,0,0,0,0,0,0,0,1
198,Digital Rights Ma0gement,0,0,0,1,0,0,0,0,0


In [38]:
print(mapping[mapping.primarySector.str.contains('0')].count())

#Replacing 0 in the beginning of the field
mapping['primarySector']  = mapping.primarySector.apply(lambda x: x.replace('0', 'Na') if (x[0] == '0') else x )

#Replacing 0 in the field anywhere else, skipping record having 2.0
mapping['primarySector']  = mapping.primarySector.apply(lambda x: x.replace('0', 'na') if ('2.0' not in x) else x )

print(mapping[mapping.primarySector.str.contains('0')].count())

primarySector                              53
Automotive & Sports                        53
Blanks                                     53
Cleantech / Semiconductors                 53
Entertainment                              53
Health                                     53
Manufacturing                              53
News, Search and Messaging                 53
Others                                     53
Social, Finance, Analytics, Advertising    53
dtype: int64
primarySector                              1
Automotive & Sports                        1
Blanks                                     1
Cleantech / Semiconductors                 1
Entertainment                              1
Health                                     1
Manufacturing                              1
News, Search and Messaging                 1
Others                                     1
Social, Finance, Analytics, Advertising    1
dtype: int64


In [39]:
print(mapping.primarySector)

1                                      3D
2                             3D Printing
3                           3D Technology
4                              Accounting
5                        Active Lifestyle
6                            Ad Targeting
7                      Advanced Materials
8                        Adventure Travel
9                             Advertising
10                  Advertising Exchanges
11                   Advertising Networks
12                  Advertising Platforms
13                                 Advice
14                              Aerospace
15                            Agriculture
16                  Air Pollution Control
17                             Algorithms
18                            All Markets
19                           All Students
20                   Alternative Medicine
21                                 Alumni
22                              Analytics
23                                Android
24                                

In [40]:
#Saving the coresponding Sector information in a column
mapping = mapping.assign(sectorColumn = lambda x : (x == 1).idxmax(axis=1))

In [41]:
mapping.sectorColumn.unique()

array(['Manufacturing', 'Social, Finance, Analytics, Advertising',
       'Health', 'Automotive & Sports', 'Others',
       'Cleantech / Semiconductors', 'News, Search and Messaging',
       'Entertainment'], dtype=object)

In [42]:

top9CountriesFrame['primarySector'] = top9CountriesFrame['category_list'].str.split("|", n=1, expand = True)[0]
print(top9CountriesFrame.shape)


mergedFrame = pd.merge(top9CountriesFrame, mapping, on = 'primarySector', how = 'left')

print(top9CountriesFrame.shape)
mergedFrame.tail(50)

mergedFrame.isnull().sum()

(44118, 14)
(44118, 14)


company_permalink                             0
funding_round_permalink                       0
funding_round_type                            0
funded_at                                     0
raised_amount_usd                             0
name                                          0
homepage_url                               2000
category_list                                 0
status                                        0
country_code                                  0
state_code                                  285
region                                      446
city                                        446
primarySector                                 0
Automotive & Sports                          11
Blanks                                       11
Cleantech / Semiconductors                   11
Entertainment                                11
Health                                       11
Manufacturing                                11
News, Search and Messaging              

In [43]:
mergedFrame.loc[mergedFrame.sectorColumn.isnull(), 'primarySector']

8353                    Natural Gas Uses
11536                 Adaptive Equipment
14069                             Racing
14348                   Specialty Retail
16996    Biotechnology and Semiconductor
20305                  Rapidly Expanding
27190                     Product Search
33956                          GreenTech
34256                         Retirement
34257                         Retirement
40716                 Adaptive Equipment
Name: primarySector, dtype: object

For 11 primary sectors, main sector could not be found from mapping frame, need to delete these rows from mergedFrame

In [44]:
mergedFrame = mergedFrame[~mergedFrame.sectorColumn.isnull()]
mergedFrame.isnull().sum()

company_permalink                             0
funding_round_permalink                       0
funding_round_type                            0
funded_at                                     0
raised_amount_usd                             0
name                                          0
homepage_url                               1998
category_list                                 0
status                                        0
country_code                                  0
state_code                                  285
region                                      446
city                                        446
primarySector                                 0
Automotive & Sports                           0
Blanks                                        0
Cleantech / Semiconductors                    0
Entertainment                                 0
Health                                        0
Manufacturing                                 0
News, Search and Messaging              

In [45]:
#Checkpoint-4 frame Expected Results: Code for a merged data frame with each primary sector mapped to its main sector 
#(the primary sector should be present in a separate column).
mergedFrame.head()
mergedFrame.columns

Index(['company_permalink', 'funding_round_permalink', 'funding_round_type',
       'funded_at', 'raised_amount_usd', 'name', 'homepage_url',
       'category_list', 'status', 'country_code', 'state_code', 'region',
       'city', 'primarySector', 'Automotive & Sports', 'Blanks',
       'Cleantech / Semiconductors', 'Entertainment', 'Health',
       'Manufacturing', 'News, Search and Messaging', 'Others',
       'Social, Finance, Analytics, Advertising', 'sectorColumn'],
      dtype='object')

Checkpoint-5 
Create three separate data frames D1, D2 and D3 for each of the three countries containing the observations of funding type FT falling within the 5-15 million USD range. The three data frames should contain:
All the columns of the master_frame along with the primary sector and the main sector

The total number (or count) of investments for each main sector in a separate column

The total amount invested in each main sector in a separate column

In [46]:
#Creating table having total amount invested in each main sector
mainSectorAmountSum = mergedFrame.pivot_table(values = 'raised_amount_usd', index = 'sectorColumn', aggfunc='sum')
mainSectorAmountSum.rename(columns = {'raised_amount_usd':'sectorSumAmount'}, inplace=True)

In [47]:
#Creating table having total number (or count) of investments for each main sector
mainSectorCount = mergedFrame.pivot_table(values = 'raised_amount_usd', index = 'sectorColumn', aggfunc='count')
mainSectorCount.rename(columns = {'raised_amount_usd':'sectorCount'}, inplace=True)

In [48]:
print(mainSectorCount.head(10))
print(mainSectorAmountSum.head(10))

                                         sectorCount
sectorColumn                                        
Automotive & Sports                              662
Cleantech / Semiconductors                      9430
Entertainment                                   2412
Health                                          3769
Manufacturing                                   3063
News, Search and Messaging                      5437
Others                                         10549
Social, Finance, Analytics, Advertising         8785
                                         sectorSumAmount
sectorColumn                                            
Automotive & Sports                             15164.45
Cleantech / Semiconductors                     135943.72
Entertainment                                   25417.60
Health                                          39452.92
Manufacturing                                   32379.37
News, Search and Messaging                      60465.60
Others        

In [49]:
mergedFrame.head()

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,name,homepage_url,category_list,status,country_code,...,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",sectorColumn
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10.00,#fame,http://livfame.com,Media,operating,IND,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Entertainment
1,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,19-03-2008,2.00,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,"News, Search and Messaging"
2,/organization/0ndine-biomedical-inc,/funding-round/954b9499724b946ad8c396a57a5f3b72,venture,21-12-2009,0.72,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,operating,CAN,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Cleantech / Semiconductors
3,/organization/0xdata,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,09-11-2015,20.00,H2O.ai,http://h2o.ai/,Analytics,operating,USA,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Social, Finance, Analytics, Advertising"
4,/organization/0xdata,/funding-round/ae2a174c06517c2394aed45006322a7e,venture,03-01-2013,1.70,H2O.ai,http://h2o.ai/,Analytics,operating,USA,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Social, Finance, Analytics, Advertising"


In [50]:
#Merge above generated 2 tables on to mergedFrame

mergedFrame.columns
mergedFrame = pd.merge(mergedFrame, mainSectorAmountSum, how='left', on = 'sectorColumn')
mergedFrame = pd.merge(mergedFrame, mainSectorCount, how='left', on = 'sectorColumn')

mergedFrame.head(50)

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,name,homepage_url,category_list,status,country_code,...,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",sectorColumn,sectorSumAmount,sectorCount
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10.00,#fame,http://livfame.com,Media,operating,IND,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Entertainment,25417.60,2412
1,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,19-03-2008,2.00,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,"News, Search and Messaging",60465.60,5437
2,/organization/0ndine-biomedical-inc,/funding-round/954b9499724b946ad8c396a57a5f3b72,venture,21-12-2009,0.72,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,operating,CAN,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Cleantech / Semiconductors,135943.72,9430
3,/organization/0xdata,/funding-round/3bb2ee4a2d89251a10aaa735b1180e44,venture,09-11-2015,20.00,H2O.ai,http://h2o.ai/,Analytics,operating,USA,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Social, Finance, Analytics, Advertising",108183.28,8785
4,/organization/0xdata,/funding-round/ae2a174c06517c2394aed45006322a7e,venture,03-01-2013,1.70,H2O.ai,http://h2o.ai/,Analytics,operating,USA,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Social, Finance, Analytics, Advertising",108183.28,8785
5,/organization/0xdata,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,19-07-2014,8.90,H2O.ai,http://h2o.ai/,Analytics,operating,USA,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Social, Finance, Analytics, Advertising",108183.28,8785
6,/organization/1-mainstream,/funding-round/b952cbaf401f310927430c97b68162ea,venture,17-03-2015,5.00,1 Mainstream,http://www.1mainstream.com,Apps|Cable|Distribution|Software,acquired,USA,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,"News, Search and Messaging",60465.60,5437
7,/organization/10-minutes-with,/funding-round/0faccbbcc5818dc5326469f13f5a8ac8,venture,09-10-2014,4.00,10 Minutes With,http://10minuteswith.com,Education,operating,GBR,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Others,109488.37,10549
8,/organization/1000memories,/funding-round/502bd0e50c27616995e4bdad24605ef8,venture,16-02-2011,2.52,1000memories,http://1000memories.com,Curated Web,acquired,USA,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,"News, Search and Messaging",60465.60,5437
9,/organization/1000museums-com,/funding-round/13be128d655076a025221d7fddc90d68,venture,14-10-2009,0.27,1000museums.com,http://www.1000museums.com,Curated Web,operating,USA,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,"News, Search and Messaging",60465.60,5437


In [51]:
mergedFrame.to_csv("./mergedFrame.csv",sep='\t', encoding='utf-8', index=False)

In [52]:
#mergedFrame.info()
D1 = mergedFrame[(mergedFrame.country_code == 'USA') & (mergedFrame.raised_amount_usd >= 5) & (mergedFrame.raised_amount_usd <= 15)]
D1.head(100)
D1.country_code.describe()
#export D1 frame
D1.to_csv("./D1.csv",sep='\t', encoding='utf-8', index=False)

In [53]:
D2 = mergedFrame[(mergedFrame.country_code == 'GBR') & (mergedFrame.raised_amount_usd >= 5) & (mergedFrame.raised_amount_usd <= 15)]
D2.head(100)
D2.country_code.describe() 
#export D2 frame
D2.to_csv("./D2.csv",sep='\t', encoding='utf-8', index=False)

In [54]:
D3 = mergedFrame[(mergedFrame.country_code == 'IND') & (mergedFrame.raised_amount_usd >= 5) & (mergedFrame.raised_amount_usd <= 15)]
D3.head(100)
D3.country_code.describe()
#export D3 frame
D3.to_csv("./D3.csv",sep='\t', encoding='utf-8', index=False)

In [55]:
mergedFrame.isnull().sum()

company_permalink                             0
funding_round_permalink                       0
funding_round_type                            0
funded_at                                     0
raised_amount_usd                             0
name                                          0
homepage_url                               1998
category_list                                 0
status                                        0
country_code                                  0
state_code                                  285
region                                      446
city                                        446
primarySector                                 0
Automotive & Sports                           0
Blanks                                        0
Cleantech / Semiconductors                    0
Entertainment                                 0
Health                                        0
Manufacturing                                 0
News, Search and Messaging              

In [56]:
print((D1.pivot_table(values = 'raised_amount_usd', index = 'sectorColumn', aggfunc='count')).sum())
print((D1.pivot_table(values = 'raised_amount_usd', index = 'sectorColumn', aggfunc='sum')).sum())
print((D1.pivot_table(values = 'raised_amount_usd', index = 'sectorColumn', aggfunc='count')))

print(D1[D1.sectorColumn == 'Others'].pivot_table(values = 'raised_amount_usd', index = 'company_permalink',aggfunc = 'sum').sort_values(by='raised_amount_usd', ascending=False).head())
print(D1[D1.sectorColumn == 'Social, Finance, Analytics, Advertising'].pivot_table(values = 'raised_amount_usd', index = 'company_permalink',aggfunc = 'sum').sort_values(by='raised_amount_usd', ascending=False).head())

#D1[D1.company_permalink == '/organization/virtustream']

raised_amount_usd    12095
dtype: int64
raised_amount_usd    108017.2
dtype: float64
                                         raised_amount_usd
sectorColumn                                              
Automotive & Sports                                    167
Cleantech / Semiconductors                            2355
Entertainment                                          591
Health                                                 914
Manufacturing                                          805
News, Search and Messaging                            1587
Others                                                2958
Social, Finance, Analytics, Advertising               2718
                                 raised_amount_usd
company_permalink                                 
/organization/virtustream                    64.30
/organization/capella                        54.97
/organization/pmw-technologies               54.89
/organization/airtight-networks              54.20
/organization/decar

In [57]:
print((D2.pivot_table(values = 'raised_amount_usd', index = 'sectorColumn', aggfunc='count')).sum())
print((D2.pivot_table(values = 'raised_amount_usd', index = 'sectorColumn', aggfunc='sum')).sum())
print((D2.pivot_table(values = 'raised_amount_usd', index = 'sectorColumn', aggfunc='count')))
print(D2[D2.sectorColumn == 'Others'].pivot_table(values = 'raised_amount_usd', index = 'company_permalink',aggfunc = 'sum').sort_values(by='raised_amount_usd', ascending=False).head())
print(D2[D2.sectorColumn == 'Social, Finance, Analytics, Advertising'].pivot_table(values = 'raised_amount_usd', index = 'company_permalink',aggfunc = 'sum').sort_values(by='raised_amount_usd', ascending=False).head())

raised_amount_usd    622
dtype: int64
raised_amount_usd    5394.07
dtype: float64
                                         raised_amount_usd
sectorColumn                                              
Automotive & Sports                                     16
Cleantech / Semiconductors                             130
Entertainment                                           56
Health                                                  24
Manufacturing                                           42
News, Search and Messaging                              73
Others                                                 148
Social, Finance, Analytics, Advertising                133
                                       raised_amount_usd
company_permalink                                       
/organization/electric-cloud                       37.00
/organization/sensage                              36.25
/organization/enigmatic                            32.50
/organization/silverrail-technologies      

In [58]:
print((D3.pivot_table(values = 'raised_amount_usd', index = 'sectorColumn', aggfunc='count')).sum())
print((D3.pivot_table(values = 'raised_amount_usd', index = 'sectorColumn', aggfunc='sum')).sum())
print((D3.pivot_table(values = 'raised_amount_usd', index = 'sectorColumn', aggfunc='count')))

print(D3[D3.sectorColumn == 'Others'].pivot_table(values = 'raised_amount_usd', index = 'company_permalink',aggfunc = 'sum').sort_values(by='raised_amount_usd', ascending=False).head())
print(D3[D3.sectorColumn == 'Social, Finance, Analytics, Advertising'].pivot_table(values = 'raised_amount_usd', index = 'company_permalink',aggfunc = 'sum').sort_values(by='raised_amount_usd', ascending=False).head())
#D1.pivot_table()
#D1.pivot_table(values = 'raised_amount_usd', index = 'sectorColumn', aggfunc='count')

raised_amount_usd    328
dtype: int64
raised_amount_usd    2949.55
dtype: float64
                                         raised_amount_usd
sectorColumn                                              
Automotive & Sports                                     13
Cleantech / Semiconductors                              20
Entertainment                                           33
Health                                                  19
Manufacturing                                           21
News, Search and Messaging                              52
Others                                                 110
Social, Finance, Analytics, Advertising                 60
                                raised_amount_usd
company_permalink                                
/organization/firstcry-com                   39.0
/organization/myntra                         38.0
/organization/commonfloor                    32.9
/organization/pepperfry-com                  28.0
/organization/itzcash-card-l